In [1]:
import tensorflow as tf
import tensorflow_text  # noqa
import tensorflow_hub as hub

In [2]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path
from collections import defaultdict

import pandas as pd
import numpy as np

In [3]:
EMB_SIZE = 768
BATCH_SIZE = 128
EMB_NAME = 'distilbert'

In [6]:
# Loading models from tfhub.dev
encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_preprocess/2")
preprocessor = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_L-6_H-768_A-12/1")

# Constructing model to encode texts into high-dimensional vectors
sentences = tf.keras.layers.Input(shape=(), dtype=tf.string, name="sentences")
encoder_inputs = preprocessor(sentences)
sentence_representation = encoder(encoder_inputs)["pooled_output"]
normalized_sentence_representation = tf.nn.l2_normalize(sentence_representation, axis=-1)  # for cosine similarity
model = tf.keras.Model(sentences, normalized_sentence_representation)
model.summary()

2022-07-23 23:49:35.850756: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 367248384 exceeds 10% of free system memory.


ValueError: Exception encountered when calling layer "keras_layer_5" (type KerasLayer).

in user code:

    File "/home/as/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow_hub/keras_layer.py", line 237, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (3 total):
        * <tf.Tensor 'inputs:0' shape=(None,) dtype=string>
        * False
        * None
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (3 total):
        * {'input_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_mask'),
     'input_word_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_word_ids')}
        * False
        * None
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (3 total):
        * {'input_mask': TensorSpe

In [ ]:
train = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'train.pkl'))
test = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'test.pkl'))

In [ ]:
def get_embedding(text: pd.Series, emb_name='') -> pd.DataFrame:
    n = len(text)
    embeddings = np.zeros(shape=(n, EMB_SIZE))
    for i in tqdm(range(0, n, BATCH_SIZE), total=n // BATCH_SIZE):
        sentences = tf.constant(text.iloc[i:i+BATCH_SIZE].tolist())
        embeddings[i:i+BATCH_SIZE, :] = model(sentences)
    embeddings = pd.DataFrame(
        embeddings, 
        columns=[f'{emb_name}_{c}' for c in range(EMB_SIZE)],
        index=text.index)
    return embeddings    

In [ ]:
train_embeddings = get_embedding(train[cfg.TEXT_COL], emb_name=EMB_NAME)
test_embeddings = get_embedding(test[cfg.TEXT_COL], emb_name=EMB_NAME)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
emb_path = os.path.join(cfg.DATA_PATH, EMB_NAME)
check_path(emb_path)

In [ ]:
train_embeddings.to_pickle(os.path.join(emb_path, 'train.pkl'))
test_embeddings.to_pickle(os.path.join(emb_path, 'test.pkl'))